# Análise de Investidores do Tesouro Direto

Neste projeto serão analisados os dados de investidores (pessoas físicas) do tesouro direto que aderiram ao programa no período de janeiro de 2020 a dezembro de 2023. A análise será baseada em dados no formato CSV, acessíveis no portal de [Dados Abertos](https://www.tesourotransparente.gov.br/ckan/dataset) disponibilizados pelo [Tesouro Nacional Transparente](https://www.tesourotransparente.gov.br/).

O Tesouro Direto é um programa do Tesouro Nacional, desenvolvido em parceria com a B3 (antiga BM&F Bovespa), que permite a venda de títulos públicos federais diretamente para pessoas físicas via internet. O objetivo principal do programa é captar recursos para financiar a dívida pública, ao mesmo tempo em que oferece uma forma acessível de investimento, com valores mínimos baixos e liquidez diária.

## Contexto do problema

Com o crescente interesse em investimentos, especialmente em tempos de volatilidade de mercado, o Tesouro Direto tem se tornado uma opção popular entre investidores de diferentes perfis. No entanto, compreender as características demográficas, regionais e comportamentais dos investidores é fundamental para otimizar estratégias de captação e retenção de investidores.

## Demanda do negócio

- Construir uma ABT para execução da análise
- ~~Perfil demográfico dos investidores~~
- ~~Padrões regionais de investimento~~
- ~~Comportamento operacional dos investidores~~

## Compreensão dos dados

Os dados disponíveis incluem um arquivo em formato _CSV_ e um arquivo _PDF_ com o dicionário de dados, a útlima atualização ocorreu em _02 de junho de 2024_. Para o projeto de análise de dados serão utilizados as informações dos investidores do período de 2020 a 2023. Esses dados foram extraídos do portal de [Dados Abertos](https://www.tesourotransparente.gov.br/ckan/dataset) disponibilizados pelo [Tesouro Nacional Transparente](https://www.tesourotransparente.gov.br/) sob o título "Investidores do Tesouro Direto". O conjunto de dados inclui informações detalhadas sobre os investidores cadastrados no programa, como data de adesão, profissão, cidade de residência, entre outros. Também há um indicador se o investidor está ou não ativo e outro que sinaliza se o investidor realizou operações nos últimos 12 meses. Cada investidor é identificado por um código único. Caso o investidor tenha cadastro em mais de uma instituição financeira, cada registro adicional é representado por uma nova linha, mantendo o mesmo código de investidor.

## Dicionário de dados

| Variáveis | Descrição | Domínios |
|------------------------------|------------------------------|------------------------------|
| Codigo do Investidor | Código identificador do investidor. | |
| Data de Adesao       | Data de adesão ao programa Tesouro Direto.<br>A data “1900” é atribuída para as pessoas jurídicas cadastradas no Programa. | Calendário |
| Estado Civil | <br>Valor “Não se aplica” é atribuído para as instituições de caridade aptas a receber doações. | Solteiro(a), Desquitado(a), Viúvo(a), Divorciado(a), Casado(a) com brasileiro(a) nato(a), Casado(a) com brasileiro(a) naturalizado(a), Casado(a) com estrangeiro(a), União estável, Separado judic., Não se aplica; |
| Genero | <br>Valor “Não se aplica” é atribuído para as instituições de caridade aptas a receber doações. | M (Masculino), F (Feminino) ou N (Não se aplica) |
| Profissao | <br>Valor “Não se aplica” é atribuído para as instituições de caridade aptas a receber doações. | |
| Idade | <br>Registros com o valor “00*” apresentam erro no cadastro. | |
| UF do Investidor |  | |
| Cidade do Investidor |  | |
| Pais do Investidor |  | |
| Situacao da Conta |  | A (Ativo), D (Desativado). |
| Operou 12 Meses |  | S (Sim), N (Não). |

## Sumário

1. **Importação de bibliotecas**
2. **Criação e iniciação de uma sessão Spark**
3. **Criação do dataset a partir da leitura do arquivo \*.csv**
4. **Análise dos dados para construção da ABT**
5. **Construção da ABT**
6. **Salvando a ABT em formato parquet**

<br>

---

<br>

### 1. Importação de bibliotecas

In [1]:
from pyspark.sql import SparkSession
import os
import time

### 2. Criação e iniciação de uma sessão Spark

In [2]:
appName = 'PySpark - ABT de Investidores do Tesouro Direto'

spark = SparkSession.builder \
    .appName(appName) \
    .config('spark.driver.memory', '8g') \
    .config('spark.driver.cores', '2') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.cores', '4') \
    .master('local[*]') \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark

### 3. Criação do dataset a partir da leitura do arquivo *.csv

In [3]:
# Caminho do arquivo CSV
caminho_csv = 'dados/Investidores_Tesouro_Direto.csv'

df = spark.read.csv(caminho_csv, encoding='ISO-8859-1', sep=';', header=True, inferSchema=True)

# Exibir o esquema do DataFrame
df.printSchema()

root
 |-- Codigo do Investidor: integer (nullable = true)
 |-- Data de Adesao: string (nullable = true)
 |-- Estado Civil: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Profissao: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- UF do Investidor: string (nullable = true)
 |-- Cidade do Investidor: string (nullable = true)
 |-- Pais do Investidor: string (nullable = true)
 |-- Situacao da Conta: string (nullable = true)
 |-- Operou 12 Meses: string (nullable = true)



In [4]:
def exibir_data_criacao_modificacao(caminho):
    # Caminho do arquivo
    arquivo = caminho

    # Verificar data de modificação e criação
    data_modificacao = os.path.getmtime(arquivo)  # Timestamp de modificação
    data_criacao = os.path.getctime(arquivo)      # Timestamp de criação

    # Converter timestamps para formato legível
    data_modificacao = time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(data_modificacao))
    data_criacao = time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(data_criacao))

    print(f'\nData de Criação:     {data_criacao}')
    print(f'\nData de Modificação: {data_modificacao}')


In [5]:
exibir_data_criacao_modificacao(caminho_csv)


Data de Criação:     08-09-2024 10:37:25

Data de Modificação: 07-05-2024 10:10:53


In [6]:
# Exibir as 20 primeiras linhas do DataFrame
df.show(truncate=False)

+--------------------+--------------+-----------------------------------+------+---------------------------------------+-----+----------------+--------------------+------------------+-----------------+---------------+
|Codigo do Investidor|Data de Adesao|Estado Civil                       |Genero|Profissao                              |Idade|UF do Investidor|Cidade do Investidor|Pais do Investidor|Situacao da Conta|Operou 12 Meses|
+--------------------+--------------+-----------------------------------+------+---------------------------------------+-----+----------------+--------------------+------------------+-----------------+---------------+
|164912              |11/12/2008    |Casado(a) com brasileiro(a) nato(a)|M     |ENGENHEIRO                             |74   |SP              |SAO PAULO           |BRASIL            |D                |N              |
|165560              |17/12/2008    |Desquitado(a)                      |M     |BANCARIO E ECONOMIÁRIO                 |39   |SP

In [7]:
# Exibindo a quantidade de linhas e colunas do dataframe

print(f'\nVOLUMETRIA\n')
print(f'Quantidade de linhas (registros):  {df.count()}')
print(f'Quantidade de colunas (variáveis): {len(df.columns)}')


VOLUMETRIA

Quantidade de linhas (registros):  51988877
Quantidade de colunas (variáveis): 11


### 4. Análise dos dados para construção da ABT

In [8]:
# Criando uma View temporária para uso do SparkSQL
df.createOrReplaceTempView('tb_investidores')

In [9]:
# Verificando a volume total de registros por ano
qtde_ano = spark.sql('''
    SELECT
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) AS ANO,
        COUNT(*) AS QTDE_TOTAL,
        CAST(COUNT(*) * 100 / SUM(COUNT(*)) OVER() AS DECIMAL(10,3)) AS QTDE_TOTAL_PERCENT
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        ANO
    ORDER BY
        ANO
''')

qtde_ano.show(24, truncate=False)

+----+----------+------------------+
|ANO |QTDE_TOTAL|QTDE_TOTAL_PERCENT|
+----+----------+------------------+
|2020|5746371   |14.667            |
|2021|12711777  |32.446            |
|2022|11758284  |30.012            |
|2023|8962042   |22.875            |
+----+----------+------------------+



In [10]:
# Verificando a volume total de registros por país
qtde_pais = spark.sql('''
    SELECT
        `Pais do Investidor`,
        COUNT(*) AS QTDE_TOTAL,
        CAST(COUNT(*) * 100 / SUM(COUNT(*)) OVER() AS DECIMAL(10,3)) AS QTDE_TOTAL_PERCENT
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        `Pais do Investidor`
    ORDER BY
        `Pais do Investidor`
''')

qtde_pais.show(truncate=False)

+------------------+----------+------------------+
|Pais do Investidor|QTDE_TOTAL|QTDE_TOTAL_PERCENT|
+------------------+----------+------------------+
|BRASIL            |39178474  |100.000           |
+------------------+----------+------------------+



In [11]:
# Verificando o top 10 da quantidade de contas ativas
top_10_contas = spark.sql('''
    SELECT
        `Codigo do Investidor`,
        COUNT(*) AS QTDE_TOTAL,
        SUM(CASE WHEN `Situacao da Conta` = 'A' THEN 1 ELSE 0 END) AS QTDE_CONTAS_ATIVAS,
        SUM(CASE WHEN `Situacao da Conta` = 'D' THEN 1 ELSE 0 END) AS QTDE_CONTAS_DESATIVADAS
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        `Codigo do Investidor`
    ORDER BY
        QTDE_CONTAS_ATIVAS DESC
    LIMIT 
        10
''')

top_10_contas.show(truncate=False)

+--------------------+----------+------------------+-----------------------+
|Codigo do Investidor|QTDE_TOTAL|QTDE_CONTAS_ATIVAS|QTDE_CONTAS_DESATIVADAS|
+--------------------+----------+------------------+-----------------------+
|27178624            |23        |23                |0                      |
|167531              |27        |20                |7                      |
|11361915            |24        |20                |4                      |
|13285332            |26        |20                |6                      |
|2261004             |21        |20                |1                      |
|18801556            |21        |20                |1                      |
|11550337            |29        |20                |9                      |
|38521987            |19        |19                |0                      |
|5849913             |23        |19                |4                      |
|4458593             |20        |19                |1                      |

In [12]:
# Verificando o top 10 da quantidade de contas desativadas
top_10_contas_desativadas = spark.sql('''
    SELECT
        `Codigo do Investidor`,
        COUNT(*) AS QTDE_TOTAL,
        SUM(CASE WHEN `Situacao da Conta` = 'A' THEN 1 ELSE 0 END) AS QTDE_CONTAS_ATIVAS,
        SUM(CASE WHEN `Situacao da Conta` = 'D' THEN 1 ELSE 0 END) AS QTDE_CONTAS_DESATIVADAS
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        `Codigo do Investidor`
    ORDER BY
        QTDE_CONTAS_DESATIVADAS DESC
    LIMIT 
        10
''')

top_10_contas_desativadas.show(truncate=False)

+--------------------+----------+------------------+-----------------------+
|Codigo do Investidor|QTDE_TOTAL|QTDE_CONTAS_ATIVAS|QTDE_CONTAS_DESATIVADAS|
+--------------------+----------+------------------+-----------------------+
|13766919            |22        |0                 |22                     |
|17968697            |18        |0                 |18                     |
|17514848            |16        |0                 |16                     |
|15670324            |16        |0                 |16                     |
|11115346            |16        |0                 |16                     |
|19249523            |16        |0                 |16                     |
|9405224             |15        |0                 |15                     |
|12606211            |15        |0                 |15                     |
|5268435             |15        |0                 |15                     |
|8803008             |14        |0                 |14                     |

In [13]:
# Particionando os registros por investidor e ranqueando por data de adesão
rank_investidor_data = spark.sql('''
    SELECT
        `Codigo do Investidor`,
        ROW_NUMBER() OVER (
            PARTITION BY `Codigo do Investidor` 
            ORDER BY TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')
        ) AS rank_data_adesao
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    ORDER BY
        `Codigo do Investidor`
    LIMIT
        15
''')

rank_investidor_data.show(truncate=False)

+--------------------+----------------+
|Codigo do Investidor|rank_data_adesao|
+--------------------+----------------+
|4                   |1               |
|5                   |1               |
|5                   |2               |
|5                   |3               |
|5                   |4               |
|9                   |1               |
|9                   |2               |
|9                   |3               |
|9                   |4               |
|9                   |5               |
|9                   |6               |
|9                   |7               |
|10                  |1               |
|10                  |2               |
|12                  |1               |
+--------------------+----------------+



In [14]:
# Verificando a quantidade de contas ativas e desativadas
qtde_contas = spark.sql('''
    SELECT
        `Situacao da Conta`,
        COUNT(*) AS QTDE,
        CAST(COUNT(*) * 100 / SUM(COUNT(*)) OVER() AS DECIMAL(10,3)) AS QTDE_TOTAL_PERCENT
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        `Situacao da Conta`
    ORDER BY
        `Situacao da Conta` DESC
''')

qtde_contas.show(truncate=False)

+-----------------+--------+------------------+
|Situacao da Conta|QTDE    |QTDE_TOTAL_PERCENT|
+-----------------+--------+------------------+
|D                |5641483 |14.399            |
|A                |33536991|85.601            |
+-----------------+--------+------------------+



In [15]:
# Verificando o top 10 da quantidade de operações em 12 meses
qtde_operou = spark.sql('''
    SELECT
        `Operou 12 Meses`,
        COUNT(*) AS QTDE,
        CAST(COUNT(*) * 100 / SUM(COUNT(*)) OVER() AS DECIMAL(10,3)) AS QTDE_TOTAL_PERCENT
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)
    GROUP BY
        `Operou 12 Meses`
    ORDER BY
        `Operou 12 Meses` DESC
''')

qtde_operou.show()

+---------------+--------+------------------+
|Operou 12 Meses|    QTDE|QTDE_TOTAL_PERCENT|
+---------------+--------+------------------+
|              S| 1755145|             4.480|
|              N|37423329|            95.520|
+---------------+--------+------------------+



### 5. Construção da ABT

In [16]:
# Selecionando os dados que irão compor a ABT
abt_investidores = spark.sql('''
    SELECT
        `Codigo do Investidor`,
        `Data de Adesao`,
        `Estado Civil`,
        `Genero`,
        `Profissao`,
        `Idade`,
        `UF do Investidor`,
        `Cidade do Investidor`,
        `Situacao da Conta`,
        `Operou 12 Meses`,
        ROW_NUMBER() OVER (
            PARTITION BY `Codigo do Investidor` 
            ORDER BY TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')
        ) AS rank_data_adesao,
        TO_DATE(CONCAT(SUBSTRING(`Data de Adesao`, 7, 4), 
               LPAD(SUBSTRING(`Data de Adesao`, 4, 2), 2, '0'), 
               '01'), 'yyyyMMdd') AS data_ref,
        CURRENT_DATE AS data_proc 
    FROM
        tb_investidores
    WHERE
        YEAR(TO_DATE(`Data de Adesao`, 'dd/MM/yyyy')) IN (2020, 2021, 2022, 2023)   
''')

abt_investidores.show(truncate=False)

+--------------------+--------------+-------------------------------------------+------+---------------------------------------+-----+----------------+--------------------+-----------------+---------------+----------------+----------+----------+
|Codigo do Investidor|Data de Adesao|Estado Civil                               |Genero|Profissao                              |Idade|UF do Investidor|Cidade do Investidor|Situacao da Conta|Operou 12 Meses|rank_data_adesao|data_ref  |data_proc |
+--------------------+--------------+-------------------------------------------+------+---------------------------------------+-----+----------------+--------------------+-----------------+---------------+----------------+----------+----------+
|31                  |05/06/2020    |Desquitado(a)                              |F     |DIRETOR DE EMPRESAS                    |40   |GO              |GOIANIA             |D                |N              |1               |2020-06-01|2024-09-23|
|31             

In [17]:
# Exibir o esquema da ABT

abt_investidores.printSchema()

root
 |-- Codigo do Investidor: integer (nullable = true)
 |-- Data de Adesao: string (nullable = true)
 |-- Estado Civil: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Profissao: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- UF do Investidor: string (nullable = true)
 |-- Cidade do Investidor: string (nullable = true)
 |-- Situacao da Conta: string (nullable = true)
 |-- Operou 12 Meses: string (nullable = true)
 |-- rank_data_adesao: integer (nullable = false)
 |-- data_ref: date (nullable = true)
 |-- data_proc: date (nullable = false)



In [18]:
# Exibindo a quantidade de linhas e colunas do dataframe

print(f'\nVOLUMETRIA\n')
print(f'Quantidade de linhas (registros):  {abt_investidores.count()}')
print(f'Quantidade de colunas (variáveis): {len(abt_investidores.columns)}')


VOLUMETRIA

Quantidade de linhas (registros):  39178474
Quantidade de colunas (variáveis): 13


### 6. Salvando a ABT em formato parquet

In [81]:
# Diretório onde os dados serão salvos
caminho_parquet = f'dados/ABT/'

# Verifica se o diretório Parquet para o ano de venda já existe
if os.path.exists(caminho_parquet):
    # Gravando os dados em formato 'parquet'
    abt_investidores.write.partitionBy('data_ref').parquet(caminho_parquet, mode='overwrite')
    # Validando a quantidade de linhas
    read_abt_investidores = spark.read.format('parquet').load(caminho_parquet)
    print(f'\nA ABT parquet tem {read_abt_investidores.count()} linhas.')
else:
    print(f'\nOcorreu um erro: o diretório "{caminho_parquet}" não existe!')



A ABT parquet tem 39178474 linhas.


In [82]:
exibir_data_criacao_modificacao(caminho_parquet)


Data de Criação:     12-09-2024 20:46:10

Data de Modificação: 12-09-2024 20:47:00
